# HW2 - Pandas and ICD-codes

### Get the data

For this assignment, we'll need to get some data! We will be using the Diabetes Dataset that is located here:

https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008#

Afterwards, unzip the folder and place the contents in a folder called `/data/` at this directory.

Make sure to add a file called `.gitignore` at the root of your directory and add the line 

```
    data/
```
to it so that it ignore any files that you place in the Data folder.

In [1]:
import pandas as pd
import numpy as np
import string
import re

In [2]:
diabetes_df = pd.read_csv('./data/diabetic_data.csv')
#adm_source_id_map = pd.read_csv('./data/admissions_source_id_map.csv')
#adm_type_id_map = pd.read_csv('./data/admission_type_id_map.csv')
#disch_disp_id_map = pd.read_csv('./data/discharge_disposition_id_map.csv')

![](data/datadictionary.png)

## Data Source [5 pts]

When we begin working with data, it is important to try and understand the data that we've been given. Often the context can tell us a lot of information about the data. In fact, often times understanding what is *not* in the data is just as critical as understanding the data itself. From the above link, the information about how the data was collected can be found in this [paper](https://www.hindawi.com/journals/bmri/2014/781670/)

List the 5 inclusion criteria to be in the dataset: 

1. It is an inpatient encounter(a hospital admission).
2. It is a "diabetic"encounter, that is, one during which any kind of diabetes was entered to the system as a diagnosis.
3. The length of stay was at least 1 day and at most 14 days.
4. Laboratory tests were performed during the encounter.
5. Medications were administered during the encounter.


## Explore the Data [55 pts]
Using the data, answer the following questions:

#### 1. How many rows does the `diabetes_df` have? How many columns? [2.5 pts]

In [3]:
diabetes_df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [4]:
diabetes_df.shape

(101766, 50)

In [5]:
print("Number of rows: {}".format(diabetes_df.shape[0]))
print("Number of columns: {}".format(diabetes_df.shape[1]))

Number of rows: 101766
Number of columns: 50


#### 2. How many unique encounters are there? How many unique patients? [2.5 pts]

In [6]:
diabetes_df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [7]:
n_encounter = diabetes_df['encounter_id'].nunique()
n_patient = diabetes_df['patient_nbr'].nunique()

In [8]:
print("Number of unique encounters: {}".format(n_encounter))
print("Number of unique patients: {}".format(n_patient))

Number of unique encounters: 101766
Number of unique patients: 71518


#### 3. What is the most amount of encounters that a single patient has in the dataset? [5 pts]

In [9]:
print("Number of most encounters: {}".format(diabetes_df['patient_nbr'].value_counts().max()))

Number of most encounters: 40


#### 4. Show the proportion of non-missing values are in the dataset for each column. [10 pts]
> Make sure you check to see if there are missing values that aren't coded as missing, but should be

In [10]:
diabetes_df = diabetes_df.replace('?', np.nan)
diabetes_df = diabetes_df.replace('None', np.nan)

In [11]:
(1 - (diabetes_df.isnull() | diabetes_df.isna()).sum() / diabetes_df.index.size).round(4)

encounter_id                1.0000
patient_nbr                 1.0000
race                        0.9777
gender                      1.0000
age                         1.0000
weight                      0.0314
admission_type_id           1.0000
discharge_disposition_id    1.0000
admission_source_id         1.0000
time_in_hospital            1.0000
payer_code                  0.6044
medical_specialty           0.5092
num_lab_procedures          1.0000
num_procedures              1.0000
num_medications             1.0000
number_outpatient           1.0000
number_emergency            1.0000
number_inpatient            1.0000
diag_1                      0.9998
diag_2                      0.9965
diag_3                      0.9860
number_diagnoses            1.0000
max_glu_serum               0.0525
A1Cresult                   0.1672
metformin                   1.0000
repaglinide                 1.0000
nateglinide                 1.0000
chlorpropamide              1.0000
glimepiride         

#### 5. For all numeric columns, show summary statistics (mean, median, max, min, etc) [2.5 pts]

In [12]:
diabetes_df.describe()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,1.017660e+05,1.017660e+05,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,1.652016e+08,5.433040e+07,2.024006,3.715642,5.754437,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,0.635566,7.422607
std,1.026403e+08,3.869636e+07,1.445403,5.280166,4.064081,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.262863,1.933600
min,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,8.496119e+07,2.341322e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,1.523890e+08,4.550514e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,2.302709e+08,8.754595e+07,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


#### 6. For all columns with a `dtype` of object, show the count of all of the values in that column [5 pts]
> You may want to reference the `pd.DataFrame.select_dtypes()` function 

In [13]:
object_df = diabetes_df.select_dtypes(include='object')
for column in object_df.columns:
    print(object_df[column].value_counts())
    print('\n')

Caucasian          76099
AfricanAmerican    19210
Hispanic            2037
Other               1506
Asian                641
Name: race, dtype: int64


Female             54708
Male               47055
Unknown/Invalid        3
Name: gender, dtype: int64


[70-80)     26068
[60-70)     22483
[50-60)     17256
[80-90)     17197
[40-50)      9685
[30-40)      3775
[90-100)     2793
[20-30)      1657
[10-20)       691
[0-10)        161
Name: age, dtype: int64


[75-100)     1336
[50-75)       897
[100-125)     625
[125-150)     145
[25-50)        97
[0-25)         48
[150-175)      35
[175-200)      11
>200            3
Name: weight, dtype: int64


MC    32439
HM     6274
SP     5007
BC     4655
MD     3532
CP     2533
UN     2448
CM     1937
OG     1033
PO      592
DM      549
CH      146
WC      135
OT       95
MP       79
SI       55
FR        1
Name: payer_code, dtype: int64


InternalMedicine                     14635
Emergency/Trauma                      7565
Family/GeneralPractice  

#### 7. What is the average number of labs administered by age category [2.5 pts]

In [14]:
diabetes_df.groupby('age')['num_lab_procedures'].mean()  #as_index=False

age
[0-10)      41.012422
[10-20)     43.096961
[20-30)     43.066385
[30-40)     43.033642
[40-50)     42.785958
[50-60)     42.611961
[60-70)     42.600632
[70-80)     43.157396
[80-90)     44.085015
[90-100)    44.695310
Name: num_lab_procedures, dtype: float64

#### 8. Does the number of diagnoses equal the number of non-NA entries in the diag_* columns? [2.5 pts]

In [15]:
((~diabetes_df.filter(regex='diag_.*').isnull()).sum(axis=1)) == diabetes_df['number_diagnoses']

0          True
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17         True
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25         True
26        False
27         True
28        False
29        False
          ...  
101736    False
101737    False
101738    False
101739    False
101740    False
101741    False
101742    False
101743    False
101744    False
101745     True
101746    False
101747    False
101748    False
101749    False
101750    False
101751    False
101752    False
101753    False
101754    False
101755    False
101756    False
101757    False
101758    False
101759    False
101760    False
101761    False
101762    False
101763    False
101764    False
101765    False
Length: 101766, dtype: b

#### 9. Create a new column that has the value of 1 if the medical specialty in that row contains the word Surgery and 0 otherwise. [10 pts]

In [16]:
new_column = diabetes_df['medical_specialty'].str.contains('Surgery', regex=True)
new_column = new_column.replace(False, 0)
new_column = new_column.replace(np.nan, 0)
new_column = new_column.replace(True, 1)
diabetes_df['Surgery'] = new_column

In [17]:
diabetes_df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,Surgery
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,NO,0
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,Up,No,No,No,No,No,Ch,Yes,>30,0
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,Yes,NO,0
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,Up,No,No,No,No,No,Ch,Yes,NO,0
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,Steady,No,No,No,No,No,Ch,Yes,NO,0


#### 10. How many encounters where the patient was between the ages of 0 and 20 took place with Pediatric providers? [5 pts]

In [18]:
diabetes_df[((diabetes_df['age'] == '[0-10)') 
             | (diabetes_df['age'] == '[10-20)')) 
            & diabetes_df['medical_specialty'].str.contains('Pediatric', regex=True)]['encounter_id'].count()

429

#### 11. Join in the admission source, admission_type_id, and discharge_disposition_id to the diabetes dataframe. Use the default inner join method [5 pts]

#### 12. find the counts of each of the available A1Cresult categories, broken down by whether or not the patient was readmitted (regardless of whether it was less than or greater than 30 days) [2.5 pts]

In [19]:
#diabetes_df['A1Cresult'] = diabetes_df['A1Cresult'].replace('>7',7)
#diabetes_df['A1Cresult'] = diabetes_df['A1Cresult'].replace('>8',8)
df2 = diabetes_df.copy()
df2['readmitted'] = df2['readmitted'].replace('<30','Yes')
df2['readmitted'] = df2['readmitted'].replace('>30','Yes')
df2[df2['A1Cresult'].notnull()].groupby(['A1Cresult','readmitted'])['A1Cresult'].count()

A1Cresult  readmitted
>7         NO            2129
           Yes           1683
>8         NO            4504
           Yes           3712
Norm       NO            2909
           Yes           2081
Name: A1Cresult, dtype: int64


## Single-level CCS categories [40 pts]

The columns `diag_1`, `diag_2`,  and `diag_3` contain ICD-9-CM codes for the encounters that took place in this dataset. However, if we count up the number of unique values between the 3 columns, we can see that the data is very sparse. 

As we discussed in class, the single-level CCS categories can be used instead to group similar ICD codes together. Download the latest version of the ICD-9-CM single-level CCS here: [https://www.hcup-us.ahrq.gov/toolssoftware/ccs/Single_Level_CCS_2015.zip](https://www.hcup-us.ahrq.gov/toolssoftware/ccs/Single_Level_CCS_2015.zip)

Unzip this and put the contents in the `data` folder along with the Diabetes dataset and read the file called `$dxref 2015.csv` into a variable. **NOTE** You must skip the first row of this `csv` file when reading it in because there is a note there that is unrelated to the contents of the data. Look up how to do this using the `pd.read_csv` function

In [20]:
dxerf_df = pd.read_csv("data/$dxref 2015.csv", skiprows=[0])
dxerf_df.head()

,'ICD-9-CM CODE','CCS CATEGORY','CCS CATEGORY DESCRIPTION','ICD-9-CM CODE DESCRIPTION','OPTIONAL CCS CATEGORY','OPTIONAL CCS CATEGORY DESCRIPTION'
0,' ','0 ','No DX',INVALID CODES IN USER DATA,' ',' '
1,'01000','1 ','Tuberculosis',PRIM TB COMPLEX-UNSPEC,' ',' '
2,'01001','1 ','Tuberculosis',PRIM TB COMPLEX-NO EXAM,' ',' '
3,'01002','1 ','Tuberculosis',PRIM TB COMPLEX-EXM UNKN,' ',' '
4,'01003','1 ','Tuberculosis',PRIM TB COMPLEX-MICRO DX,' ',' '


Examine the contents of the data. Please bear in mind that Jupyter notebooks do not render whitespace or quotes very well sometimes, so watch out for that. Make sure you examine column names with `.columns` instead of just calling `.head()` and visually inspecting, for example.

In [21]:
dxerf_df.columns

Index([''ICD-9-CM CODE'', ''CCS CATEGORY'', ''CCS CATEGORY DESCRIPTION'',
       ''ICD-9-CM CODE DESCRIPTION'', ''OPTIONAL CCS CATEGORY'',
       ''OPTIONAL CCS CATEGORY DESCRIPTION''],
      dtype='object')

If you notice, the ICD codes are not very well-formatted in either the Diabetes dataset (for example, the code `8` should really be `008.0`) or the Single-level CCS crosswalk (`' and whitespace characters`). 

This is quite typical of healthcare data, unfortunately. Many of the publically available files are not suited for reading into modern programming languages. Often, they are limited to SAS format datasets, which is a proprietary software suite for working with statistical packages that is ubiquitous in health care.

In order to use the CCS groupings, we'll have to clean both the groupings *and* the diabetes data as well. Here is the general procedure that we will take (Although this is an imperfect mapping as well).

**Remove all quotes and extra whitespace from the codes and the column names in the CCS crosswalk [10 pts]**

In [22]:
dxerf_df.columns = dxerf_df.columns.str.replace('\'', '')
dxerf_df.columns = dxerf_df.columns.str.strip()
dxerf_df = dxerf_df.apply(lambda x: x.str.replace('\'', ''))
dxerf_df = dxerf_df.apply(lambda x: x.str.strip())
dxerf_df

,ICD-9-CM CODE,CCS CATEGORY,CCS CATEGORY DESCRIPTION,ICD-9-CM CODE DESCRIPTION,OPTIONAL CCS CATEGORY,OPTIONAL CCS CATEGORY DESCRIPTION
0,,0,No DX,INVALID CODES IN USER DATA,,
1,01000,1,Tuberculosis,PRIM TB COMPLEX-UNSPEC,,
2,01001,1,Tuberculosis,PRIM TB COMPLEX-NO EXAM,,
3,01002,1,Tuberculosis,PRIM TB COMPLEX-EXM UNKN,,
4,01003,1,Tuberculosis,PRIM TB COMPLEX-MICRO DX,,
5,01004,1,Tuberculosis,PRIM TB COMPLEX-CULT DX,,
6,01005,1,Tuberculosis,PRIM TB COMPLEX-HISTO DX,,
7,01006,1,Tuberculosis,PRIM TB COMPLEX-OTH TEST,,
8,01010,1,Tuberculosis,PRIM TB PLEURISY-UNSPEC,,
9,01011,1,Tuberculosis,PRIM TB PLEURISY-NO EXAM,,


Next, we're going to write a function that cleans up the ICD codes found in the Diabetes dataset. If you'll notice, there are no decimal points in the single-level CCS crosswalk. Therefore, we must make sure that our data matches that as well. Implement the function below, and we will use it in an `.apply()` call to modify the `diag_` columns.

#### Implement this function [15 pts]

In [23]:
def clean_diabetes_code(icd_code):
    """
    Formats codes found in the Diabetes dataset to be like those found in the CCS crosswalk
    
    
    If an icd_code has a decimal, remove the decimal
    
    if it has less than 3 digits, prepend '0's, until it is 3 digits.
    
    If it has 3 digits (before or after the above step), append a '0'
    
    Examples:
        250.13 -> 25013
        32 -> 0320
        315 -> 3150
        
    Args: 
        icd_code:
    
    Returns:
        formatted_code string: A formatted ICD-Code string according to the above criteria:
        
        
    """
    code_copy = str(icd_code) # Convert to string due to weird .apply behavior in Series
    if code_copy == 'nan':
        return np.nan
    
    ### Your Code here: -- modify code_copy until it returns the reformatted code. Store this in formatted_code and return it
    else:
        formatted_code = code_copy.replace('.','')
        if formatted_code.isdigit():
            while len(formatted_code)<3 :
                formatted_code = '0' + formatted_code
        if len(formatted_code)==3 :
            formatted_code = formatted_code + '0'
        return formatted_code
            

Now, replace all three `diag_` columns by calling .apply with this function. For example:

`diabetes_df['diag_1'] = diabetes_df['diag_1'].apply(clean_diabetes_code)`

In [24]:
diabetes_df['diag_1'] = diabetes_df['diag_1'].apply(clean_diabetes_code)
diabetes_df['diag_2'] = diabetes_df['diag_2'].apply(clean_diabetes_code)
diabetes_df['diag_3'] = diabetes_df['diag_3'].apply(clean_diabetes_code)

#### Join in the single-level CCS crosswalk and answer the following question: [15 pts]

In [25]:
merge_1 = pd.merge(diabetes_df, dxerf_df, left_on='diag_1', right_on='ICD-9-CM CODE')
merge_2 = pd.merge(diabetes_df, dxerf_df, left_on='diag_2', right_on='ICD-9-CM CODE')
merge_3 = pd.merge(diabetes_df, dxerf_df, left_on='diag_3', right_on='ICD-9-CM CODE')

List the top 10 condition categories (`CCS CATEGORY DESCRIPTION`) when you add up all instances over all 3 columns

In [26]:
value_counts = merge_1['CCS CATEGORY DESCRIPTION'].value_counts() + merge_2['CCS CATEGORY DESCRIPTION'].value_counts() + merge_3['CCS CATEGORY DESCRIPTION'].value_counts()
value_counts.nlargest(10)

Htn complicn    18653.0
chf;nonhp       18101.0
Coron athero    17602.0
Fluid/elc dx    13816.0
Dysrhythmia     12762.0
DiabMel w/cm    10266.0
UTI              7039.0
Anemia           5068.0
COPD             4867.0
Coma/brn dmg     4844.0
Name: CCS CATEGORY DESCRIPTION, dtype: float64